In [2]:
import numpy
import rioxarray
import xarray
import geopandas
import pathlib
import netCDF4

# Full example of creating a GeoFabrics style xarray Dataset

In [9]:
origin = [10, 35]
size = [1, 10000]
resolutions = [5, 10]
crs = 2193

for resolution in resolutions:
    x = numpy.arange(origin[0],
                     origin[0] + resolution * size[0],
                        resolution)
    y = numpy.arange(origin[1] + resolution * size[1],
                     origin[1],
                     -resolution) # following convention of GDAL where y starts at the largest
    method = ['idw', 'mean', 'median']
    elevation = numpy.random.rand(len(method), len(x), len(y))
    roughness = numpy.random.rand(len(x), len(y))
    category = numpy.random.randint(0, 4, (len(x), len(y)))

    example_xarray = xarray.Dataset(
        data_vars=dict(z=(["method", "x", "y"], elevation, {"units": "m", "vertical_datum": "EPSG:7839", "long_name": "ground elevation"}),
                       zo=(["x", "y"], roughness, {"units": "", "long_name": "roughness length"}),
                       source_class=(["x", "y"], category, {"units": "int", "long_name": "Source data classification",
                                                            "Classifications": "{'LiDAR': 1, 'ocean bathymetry': 2, 'rivers': 3, 'drains': 4, 'reference DEM': 5, 'interpolated': 0, 'no data': -1}"})),
        coords=dict(
            x=(["x"], x, {"long_name": "Easting", "units": "m"}),
            y=(["y"], y, {"long_name": "Northing", "units": "m"}),
            method=(['method'], method, {"long_name": "LiDAR interpolation"})),
    attrs= {"title": "GeoFabrics representing elevation and roughness",
            "source": "GeoFabrics version 0.8.1",
            "history": "2022-04-20T23:42:52.525023+00:00: GeoFabrics:LidarDemGenerator resolution 10;",
            "description": f"GeoFabrics:LidarDemGenerator resolution {resolution}",
            "geofabrics_instructions": "{'instructions': {'output': {'crs': {'horizontal': 2193, 'vertical': ....}}"})
        
    # Ensure the CRS and transform are respectively written in CF 
    # complient manner, and 'where GDAL can read it'
    example_xarray.rio.write_crs(crs, inplace=True)
    example_xarray.z.rio.write_crs(crs, inplace=True)
    example_xarray.zo.rio.write_crs(crs, inplace=True)
    example_xarray.source_class.rio.write_crs(crs, inplace=True)
    example_xarray.rio.write_transform(inplace=True)
    # Write to a grouped netCDF file and a stand alone file for just this grouping
    '''mode = "a" if (base_path / "grouped_data.nc").exists() else "w"
    example_xarray.to_netcdf(base_path / "grouped_data.nc",
                             group=f"/resolution_{resolution}",
                             mode=mode,
                             engine="netcdf4")
    example_xarray.to_netcdf(base_path / f"data_resolution_{resolution}.nc")'''

In [10]:
example_xarray.source_class

<xarray.DataArray 'source_class' (x: 1, y: 10000)>
array([[3, 1, 1, ..., 3, 3, 2]])
Coordinates:
  * x            (x) int32 10
  * y            (y) int32 100035 100025 100015 100005 99995 ... 85 75 65 55 45
    spatial_ref  int32 0
Attributes:
    units:            int
    long_name:        Source data classification
    Classifications:  {'LiDAR': 1, 'ocean bathymetry': 2, 'rivers': 3, 'drain...

# Small examples of creating an xarray from numpy arrays

In [2]:
base_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file")

In [ ]:
raster_origin = [10, 35]
raster_size = [200, 100]
resolution = 10
crs = 2193

x_coords = numpy.arange(raster_origin[0], raster_origin[0] + resolution * raster_size[0], resolution)
y_coords = numpy.arange(raster_origin[1] + resolution * raster_size[1],
                        raster_origin[1], -resolution) # following convention of GDAL where y starts at the largest
z_array = numpy.random.rand(1, len(y_coords), len(x_coords)) # 1 is the band dimension

example_xarray = xarray.DataArray(z_array,
                                  coords = {'band': [1], 'x': x_coords, 'y': y_coords},
                                  dims = ['band', 'y','x'], 
                                  attrs= {'scale_factor': 1.0,
                                          'add_offset': 0.0,
                                          'long_name': 'idw',
                                          '_FillValue': numpy.nan})
example_xarray.rio.write_crs(crs, inplace=True)
example_xarray.rio.write_transform(inplace=True)
#example_xarray.to_netcdf(base_path / "data_array_ex.nc")

In [ ]:
print(example_xarray.rio.transform(), '\n', example_xarray.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "data_array_ex.nc") as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)
#ex_data = ex_data.copy(deep=True)

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "data_array_ex.nc", masked=True) as df:
    df.load()
    print(df.rio.transform(), '\n', df.rio.crs)
#df = df.copy(deep=True)

### How to load in crs using gdal and pyproj

In [ ]:
import osgeo.gdal
import pyproj
ds_gdal = osgeo.gdal.Open(str(base_path / "data_array_ex.nc"))
pyproj.CRS.from_string(ds_gdal.GetProjection())

# Example from Cyprien

In [ ]:
ex_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file") \
    / 'Adaptdemo_coarse.nc'

In [ ]:
with rioxarray.rioxarray.open_rasterio(ex_path, masked=True) as ex_data:
    ex_data.load()
ex_data = ex_data.copy(deep=True)

In [ ]:
print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
ex_data

# Creating a DataSet from numpy arrays

In [32]:
ds = xarray.Dataset(
        data_vars=dict(z=(["method", "x", "y"], numpy.random.rand(2, 3, 3))),
        coords=dict(
            x=(["x"], [0,1,2]),
            y=(["y"], [0,1,2]),
            method=(['method'], ['idw', 'mean'])))

In [34]:
ds.loc[dict(method="idw")]

<xarray.Dataset>
Dimensions:  (x: 3, y: 3)
Coordinates:
  * x        (x) int32 0 1 2
  * y        (y) int32 0 1 2
    method   <U4 'idw'
Data variables:
    z        (x, y) float64 0.09187 0.2918 0.6985 ... 0.4858 0.4321 0.2598

In [33]:
ds.z.loc[dict(method="idw")]

<xarray.DataArray 'z' (x: 3, y: 3)>
array([[0.09187466, 0.29179225, 0.69847419],
       [0.644111  , 0.50498397, 0.21134968],
       [0.48581192, 0.43210526, 0.25978677]])
Coordinates:
  * x        (x) int32 0 1 2
  * y        (y) int32 0 1 2
    method   <U4 'idw'

In [14]:
example_xarray.z.data[0]

array([[0.31664013, 0.25959895, 0.19891735, ..., 0.74803607, 0.00546465,
        0.69420238],
       [0.14912444, 0.27649032, 0.46087056, ..., 0.36953865, 0.82095736,
        0.46838166],
       [0.32114326, 0.05590747, 0.58714331, ..., 0.97331251, 0.07251207,
        0.99607609],
       ...,
       [0.93574351, 0.46728678, 0.86034793, ..., 0.00209699, 0.85409134,
        0.14945992],
       [0.73864337, 0.36016072, 0.86667285, ..., 0.74314771, 0.57812378,
        0.38611096],
       [0.28716614, 0.78125957, 0.53658909, ..., 0.78661157, 0.36355173,
        0.65461543]])

In [ ]:
example_xarray.coords['spatial_ref']

In [ ]:
example_xarray.coords['x']

In [ ]:
example_xarray.rio.clip()

In [ ]:
print(example_xarray.rio.transform(), '\n', example_xarray.rio.crs)

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "data_resolution_5.nc", masked=True) as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
ex_data

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "grouped_data.nc", group="/resolution_10", masked=True) as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
ex_data

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_5") as ds_5:
    #print(ds_5.rio.transform(), '\n', ds_5.rio.crs)
    #ds_5.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()
    #print(ds_5.rio.transform(), '\n', ds_5.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_5") as ds_5:
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)
    ds_5.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_10") as ds_10:
    ds_10.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_10.roughness.plot()
    print(ds_10.rio.transform(), '\n', ds_10.rio.crs)

In [ ]:
import osgeo.gdal
import pyproj
#ds_gdal = osgeo.gdal.Open(str(base_path / "data_resolution_5.nc"))
#crs=pyproj.CRS.from_string(ds_gdal.GetProjection())

with xarray.open_dataset(base_path / "data_resolution_5.nc") as ds_5:
    
    #ds_5.rio.set_crs(crs, inplace=True);
    ds_5.zo.plot()
    #print(ds_5.rio.transform(), '\n', ds_5.rio.crs)

In [ ]:
ds_5